In [1]:
import os
import csv
import sqlite3
import gzip
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
def get_connection(data_path):
    db_path = os.path.join(data_path, 'movies.sqlite')
    conn = sqlite3.connect(db_path)
    return conn

In [3]:
def get_cursor(conn):
    cursor = conn.cursor()
    return cursor

In [4]:
input_data_path = '/home/schart/Flatiron/DataScience/Phase1/Project/Movie_Analysis/zippedData'
data_path = '/home/schart/Flatiron/DataScience/Phase1/Project/Movie_Analysis/data/'

In [5]:
conn = get_connection(data_path=data_path)
cur = get_cursor(conn)

In [6]:
def create_table(table_name, columns_string, cur):
    drop_query = f"""
    DROP TABLE IF EXISTS {table_name};
    """
    create_query = f"""
    CREATE TABLE {table_name} {columns_string};
    """
    cur.execute(drop_query)
    cur.execute(create_query)

In [7]:
def create_table_titles(cur):
    titles_columns_string = """(
        tconst TEXT PRIMARY KEY,
        primary_title TEXT,
        start_year INTEGER
        )
    """
    create_table(table_name='titles', columns_string=titles_columns_string, cur=cur)

In [8]:
def create_table_runtimes(cur):
    runtimes_columns_string = """(
        tconst TEXT PRIMARY KEY,
        runtime_minutes INTEGER,
        FOREIGN KEY(tconst) REFERENCES titles(tconst)
    )
    """
    create_table(table_name='runtimes', columns_string=runtimes_columns_string, cur=cur)

In [9]:
def create_table_genres(cur):
    genres_columns_string = """(
        tconst TEXT PRIMARY KEY,
        Action INTEGER DEFAULT 0,
        Adult INTEGER DEFAULT 0,
        Adventure INTEGER DEFAULT 0,
        Animation INTEGER DEFAULT 0,
        Biography INTEGER DEFAULT 0,
        Comedy INTEGER DEFAULT 0,
        Crime INTEGER DEFAULT 0,
        Documentary INTEGER DEFAULT 0,
        Drama INTEGER DEFAULT 0,
        Family INTEGER DEFAULT 0,
        Fantasy INTEGER DEFAULT 0,
        Game_Show INTEGER DEFAULT 0,
        History INTEGER DEFAULT 0,
        Horror INTEGER DEFAULT 0,
        Music INTEGER DEFAULT 0,
        Musical INTEGER DEFAULT 0,
        Mystery INTEGER DEFAULT 0,
        News INTEGER DEFAULT 0,
        Reality_TV INTEGER DEFAULT 0,
        Romance INTEGER DEFAULT 0,
        Sci_Fi INTEGER DEFAULT 0,
        Short INTEGER DEFAULT 0,
        Sport INTEGER DEFAULT 0,
        Talk_Show INTEGER DEFAULT 0,
        Thriller INTEGER DEFAULT 0,
        War INTEGER DEFAULT 0,
        Western INTEGER DEFAULT 0,
        FOREIGN KEY(tconst) REFERENCES titles(tconst)
    )
    """
    create_table(table_name='genres', columns_string=genres_columns_string, cur=cur)

In [10]:
def write_row(table_name, row_dict, cur):
    write_query = f"""
    INSERT INTO {table_name} ({', '.join(row_dict.keys())})
    VALUES {tuple(row_dict.values())};
    """
    cur.execute(write_query)

In [11]:
def make_row_dict(row, key_list):
    row_dict = dict()
    for key in key_list:
        row_dict[key] = row[key]
    return row_dict

In [12]:
def get_file_path(input_data_path, file_name):
    file_path = os.path.join(input_data_path, file_name)
    return file_path

In [13]:
def write_row_title(row, cur):
        title_row_list = ['tconst', 'primary_title', 'start_year']
        title_row_dict = make_row_dict(row=row, key_list=title_row_list)
        title_row_dict['start_year'] = int(title_row_dict['start_year'])
        write_row(table_name='titles', row_dict=title_row_dict, cur=cur)

In [14]:
def write_row_runtime(row, cur):
    runtime_row_list = ['tconst', 'runtime_minutes']
    runtime_row_dict = make_row_dict(row, runtime_row_list)
    if len(runtime_row_dict['runtime_minutes']) > 0:
        runtime_row_dict['runtime_minutes'] = int(runtime_row_dict['runtime_minutes'])
        write_row(table_name='runtimes', row_dict=runtime_row_dict, cur = cur)

In [15]:
def write_row_genre(row, cur):
    genres_string = row['genres']
    if len(genres_string) > 0:
        genres_list = genres_string.split(',')
        genres_list = [genre.replace('-','_') for genre in genres_list]
        genre_row_dict = dict()
        genre_row_dict['tconst'] = row['tconst']
        for genre in genres_list:
            genre_row_dict[genre] = 1
        write_row(table_name='genres', row_dict=genre_row_dict, cur=cur)

In [35]:
def read_title_basics(cur):
    file_path = get_file_path(input_data_path=input_data_path, file_name='imdb.title.basics.csv.gz')
    create_table_titles(cur=cur)
    create_table_runtimes(cur=cur)
    create_table_genres(cur=cur)
    with gzip.open(file_path, mode='rt') as f:
        reader = csv.DictReader(f)
        for row in reader:
            write_row_title(row=row, cur=cur)
            write_row_runtime(row=row, cur=cur)
            write_row_genre(row=row, cur=cur)

In [36]:
def create_table_ratings(cur):
    ratings_column_string ="""(
        tconst TEXT PRIMARY KEY,
        averagerating REAL,
        numvotes INTEGER,
        FOREIGN KEY(tconst) REFERENCES titles(tconst)
    )
    """
    create_table(table_name='ratings', columns_string=ratings_column_string, cur=cur)

In [37]:
def write_row_rating(row, cur):
    rating_row_list = ['tconst', 'averagerating', 'numvotes']
    rating_row_dict = make_row_dict(row=row, key_list=rating_row_list)
    rating_row_dict['averagerating'] = float(rating_row_dict['averagerating'])
    rating_row_dict['numvotes'] = int(rating_row_dict['numvotes'])
    write_row(table_name='ratings', row_dict=rating_row_dict, cur=cur)

In [38]:
def read_title_ratings(cur):
    file_path = get_file_path(input_data_path=input_data_path, file_name='imdb.title.ratings.csv.gz')
    create_table_ratings(cur=cur)
    with gzip.open(file_path, mode='rt') as f:
        reader = csv.DictReader(f)
        for row in reader:
            write_row_rating(row=row, cur=cur)

In [39]:
def create_table_budgets(cur):
    budgets_column_string = """(
        id INTEGER PRIMARY KEY,
        primary_title TEXT,
        start_year INTEGER,
        release_date TEXT,
        production_budget INTEGER,
        domestic_gross INTEGER,
        worldwide_gross INTEGER,
        FOREIGN KEY (primary_title, start_year) REFERENCES titles(primary_title, start_year)
    )
    """
    create_table(table_name='budgets', columns_string=budgets_column_string, cur=cur)

In [40]:
def write_row_budget(row, cur):
    budget_row_list = ['release_date',
                       'production_budget',
                       'domestic_gross',
                       'worldwide_gross'
                      ]
    budget_row_dict = make_row_dict(row=row, key_list=budget_row_list)
    budget_row_dict['production_budget'] = int(budget_row_dict['production_budget'].replace('$','').replace(',',''))
    budget_row_dict['domestic_gross'] = int(budget_row_dict['domestic_gross'].replace('$','').replace(',',''))
    budget_row_dict['worldwide_gross'] = int(budget_row_dict['worldwide_gross'].replace('$','').replace(',',''))
    budget_row_dict['primary_title'] = row['movie']
    budget_row_dict['start_year'] = datetime.strptime(row['release_date'], '%b %d, %Y').year
    write_row(table_name='budgets', row_dict=budget_row_dict, cur=cur)

In [43]:
def read_budgets(cur):
    file_path = get_file_path(input_data_path=input_data_path, file_name='tn.movie_budgets.csv.gz')
    create_table_budgets(cur=cur)
    with gzip.open(file_path, mode='rt') as f:
        reader = csv.DictReader(f)
        for row in reader:
            write_row_budget(row=row, cur=cur)

In [44]:
read_title_basics(cur)
read_title_ratings(cur)
read_budgets(cur)

In [45]:
cur.close()

In [46]:
conn.close()